<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Identifying-the-data-view-to-use-for-your-reports" data-toc-modified-id="Identifying-the-data-view-to-use-for-your-reports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Identifying the data view to use for your reports</a></span></li></ul></li><li><span><a href="#Creating-a-request-for-a-report" data-toc-modified-id="Creating-a-request-for-a-report-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating a request for a report</a></span></li><li><span><a href="#Requesting-the-data" data-toc-modified-id="Requesting-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Requesting the data</a></span><ul class="toc-item"><li><span><a href="#Breakdown" data-toc-modified-id="Breakdown-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Breakdown</a></span></li></ul></li><li><span><a href="#Complex-data-extract" data-toc-modified-id="Complex-data-extract-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Complex data extract</a></span></li><li><span><a href="#External-Tools" data-toc-modified-id="External-Tools-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>External Tools</a></span></li></ul></div>

Extracting data from Customer Journey Analytics is coming as a feature thanks to the CJA API.\
We will introduce the way to extract data in Customer Journey Analytics.\
Then we will apply some data analysis and data visualization that are not yet possible in CJA.

importing cjapy, pandas for a start and importing the configuration file. 

# Setup 

In [ ]:
import cjapy
import pandas as pd
cjapy.importConfigFile('myconfig.json')

## Identifying the data view to use for your reports

You need to know which data view to pull the data from.\
Basde on the data view selection, we will be able to extract data by placing a request for certain information.\
The possibility to create request via cjapy is simplify thanks to the `RequestCreator`

In [ ]:
cja = cjapy.CJA()

Extracting the id based on the name of the dataview

In [ ]:
dataviews = cja.getDataViews()
dv_id = dataviews.at[dataviews[dataviews['name']=='Adobe Store - Prod (CJA Lab L123 Adobe Summit 2025)'].index[0],'id']
dv_id

Extracting the dimensions related to that dataview: 

In [ ]:
dimensions = cja.getDimensions(dv_id,full=True)
metrics = cja.getMetrics(dv_id,full=True)

In [ ]:
dimensions[dimensions['id'].str.contains('product')]

In [ ]:
metrics.sample(4)

# Creating a request for a report

`cjapy` provides a module named RequestCreator that simplify the creation of data request from CJA. 

we start by instantiating a report

In [ ]:
myReportRequest = cjapy.RequestCreator()

I will define on my request:
* a data view ID 
* the time frame that is being used for the report
* the metrics that I want to extract
* the dimension I want to extract

In [ ]:
myReportRequest.setDataViewId(dv_id)
myReportRequest.setDateRange('2025-01-01','2025-01-31')
myReportRequest.addMetric('metrics/orders')
myReportRequest.addMetric('metrics/visits')
myReportRequest.setDimension('variables/product_category')

You can always check the definition of your request by realizing a simple request in notebook

In [ ]:
myReportRequest

or by using the `to_dict()` method

In [ ]:
myReportRequest.to_dict()

# Requesting the data

Once your data request is ready, you can pass the object directly on the `getReport` method.\
This will return a complex object that we will analyze

In [ ]:
myreport = cja.getReport(myReportRequest)

The report will return a `Workspace` object that contains a dataframe attribute.\
It contains all data related to your request.

In [ ]:
myreport.dataframe.head()

You can also get the request that has been sent by using `dataRequest`

In [ ]:
myreport.dataRequest

you can see how many pages has been requested, the `pageRequest` will return the maximum of page returned.\
Each page is 50 K rows

In [ ]:
myreport.pageRequested

You can get the context of the data by using either:
* globalFilters
* settings
* summaryData

In [ ]:
myreport.globalFilters

In [ ]:
myreport.settings

In [ ]:
myreport.summaryData

## Breakdown

If you wish to realize a break down of the category per the type of product.\
You can directly call the breakdown method from the response.\
You will use the index of the response (for the row to break down) and the id of the variable you want to break down with and the number of element you want. 

In [ ]:
breakdownProdName = myreport.breakdown('Book','variables/product_name',n_results=20)

It will return another workspace response.

In [ ]:
breakdownProdName.dataframe

# Complex data extract

cjapy offers method to extract data that are not easily doable via the UI.\
The `getPersonProfiles` is a method to retrieves a dataset where each row represents a person profile. This is particularly useful for extracting feature metrics and target metrics for supervised learning applications or customer segmentation analysis.

In [ ]:
feature_metrics = ["metrics/occurrences","metrics/cart_additions"]
df_profiles = cja.getPersonProfiles(
    dataviewId=dv_id,
    featureMetrics=feature_metrics,
    targetMetric="metrics/orders",
    startDate="2025-01-01",
    endDate="2025-01-31",
    sampleSize=1000,
    binaryTargetMetric=True
)

In [ ]:
df_profiles.sample(10)

# External Tools

With all the advanced capability of CJA, one of the most useful is to be able to plot and have a visual representation of the data we are seeing.\
It may help to better understand and comprehend.\
We will see advanced some advanced visualization and forecasting later, but now, descriptive visualization can be shown with more plotting tools and can complement the CJA UI. 

In [ ]:
filterId_order = "sD0F83C645C5E1CC60A495CB3@AdobeOrg_66be408dbe6ba704b6c2d506"
filterId_noOrder = "sD0F83C645C5E1CC60A495CB3@AdobeOrg_679f95f0c0f82b0d0d8d1643"
filterId_CartNoOrder = "sD0F83C645C5E1CC60A495CB3@AdobeOrg_679f98764564ad1e6222f57e"

In [ ]:
request_data = cjapy.RequestCreator()

In [ ]:
request_data.setDataViewId(dv_id)
request_data.setDateRange(start_date='2025-01-01',end_date='2025-01-31')
request_data.addMetric("metrics/occurrences")
request_data.addMetric("metrics/visits")
request_data.addMetric("metrics/visitors")
request_data.addMetric("metrics/cart_additions")
request_data.addMetric("metrics/orders")
request_data.addMetricFilter(metricId='all',filterId="sD0F83C645C5E1CC60A495CB3@AdobeOrg_66be408dbe6ba704b6c2d506",staticRow=True)
request_data.addMetricFilter(metricId='all',filterId="sD0F83C645C5E1CC60A495CB3@AdobeOrg_679f95f0c0f82b0d0d8d1643",staticRow=True)
request_data.addMetricFilter(metricId='all',filterId="sD0F83C645C5E1CC60A495CB3@AdobeOrg_679f98764564ad1e6222f57e",staticRow=True)

In [ ]:
print(request_data)

In [ ]:
data = cja.getReport(request_data)

In [ ]:
data.dataframe

In [ ]:
dataResult = data.dataframe.copy()

In [ ]:
import math
from bokeh.plotting import figure, show,output_notebook
from bokeh.transform import factor_cmap, factor_mark

In [ ]:
data = cja.getReport(request_data)

In [ ]:
output_notebook()

In [ ]:
p = figure(title = "Filter comparison for Cart Addition and Visits", background_fill_color="#fafafa")
p.xaxis.axis_label = 'Visits'
p.yaxis.axis_label = 'Cart Addition'

In [ ]:
dataResult['sizeDots'] = dataResult['metrics/visitors']/dataResult['metrics/visitors'].sum()*100
colors = ["lightskyblue","peru","orangered"]

In [ ]:
from bokeh.plotting import figure, show

p = figure()
for index, row in dataResult.iterrows():
    p.scatter(x=row["metrics/visits"], y=row["metrics/cart_additions"],
              size=row["sizeDots"], fill_color=colors[index],
              line_color=colors[index], legend_label=row['FilterName'])

p.legend.location = "top_right"
p.legend.title = "Filter Names"
show(p)
